In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import best_subset as bs_pkgs


In [3]:
import numpy as np
import pandas as pd

import  best_subset as bs1  
import best_subset2 as bs2
import best_subset_force_memoization as bs3
import best_subset_force_drop_score2 as bs4
import best_subset_force_paralell as bs5
import best_subset2_bfs as bs6
import best_subset2_bfs_v2 as bs7
import best_subset2_bfs_v3 as bs8

np.set_printoptions(suppress=True)

In [4]:


def create_synthetic_data(seed=42, n=50000, p=15):
    """
    Creates a DataFrame X of shape (n, p+1) with columns:
      - 'const': all ones (intercept)
      - 'x1', 'x2', ... 'x15'
    And a Series y with binary (0/1) outcomes drawn from a logistic model.
    
    Some of the 15 features have nonzero coefficients, others are zero,
    so there's meaningful signal to detect in a logistic regression.
    """

    np.random.seed(seed)

    # 1) Generate random features ~ N(0,1)
    X_base = np.random.randn(n, p)
    
    # 2) Define "true" coefficients
    #    For instance, let's say 5 features matter:
    #    x1, x2, x3, x4, x5 have some nonzero betas.
    #    The remaining x6..x15 have 0 effect.
    betas_true = np.array([1.5, -2.0, 0.75, 1.25, -0.5] + [0]*(p-5))
    #     -> length = 15
    
    # 3) Linear predictor: X_base dot betas_true
    #    shape -> (n, )
    lin_pred = X_base.dot(betas_true)
    
    # 4) Convert linear predictor to probability via logistic function
    #    p_i = 1 / (1 + exp(-lin_pred))
    prob = 1.0 / (1.0 + np.exp(-lin_pred))
    
    # 5) Draw binary outcomes y from Bernoulli(prob)
    y_vals = np.random.binomial(1, prob)
    
    # 6) Create a DataFrame for features, plus an intercept column
    df = pd.DataFrame(X_base, columns=[f"x{i+1}" for i in range(p)])
    df.insert(0, "const", 1.0)  # intercept
    
    # 7) Create a Series for y
    y = pd.Series(y_vals, name="y")
    
    return df, y

# Example usage
if __name__ == "__main__":
    df, y = create_synthetic_data(p=15)
    print(df.head())
    print(y.head())
    print("df shape:", df.shape)
    print("y shape:", y.shape)

   const        x1        x2        x3        x4        x5        x6  \
0    1.0  0.496714 -0.138264  0.647689  1.523030 -0.234153 -0.234137   
1    1.0 -0.562288 -1.012831  0.314247 -0.908024 -1.412304  1.465649   
2    1.0 -0.601707  1.852278 -0.013497 -1.057711  0.822545 -1.220844   
3    1.0 -0.719844 -0.460639  1.057122  0.343618 -1.763040  0.324084   
4    1.0 -0.479174 -0.185659 -1.106335 -1.196207  0.812526  1.356240   

         x7        x8        x9       x10       x11       x12       x13  \
0  1.579213  0.767435 -0.469474  0.542560 -0.463418 -0.465730  0.241962   
1 -0.225776  0.067528 -1.424748 -0.544383  0.110923 -1.150994  0.375698   
2  0.208864 -1.959670 -1.328186  0.196861  0.738467  0.171368 -0.115648   
3 -0.385082 -0.676922  0.611676  1.031000  0.931280 -0.839218 -0.309212   
4 -0.072010  1.003533  0.361636 -0.645120  0.361396  1.538037 -0.035826   

        x14       x15  
0 -1.913280 -1.724918  
1 -0.600639 -0.291694  
2 -0.301104 -1.478522  
3  0.331263  0.97554

In [7]:
# candidates = df.columns[1:].tolist()
g, Is, X, y, candidates, ordered_candidates =  prepare_step(df, y, candidates)

In [103]:
# print(np.round(Is[1,:],2))
# print(np.round(g[:],2))
candidates = df.columns.tolist()
results =  best_subset_exhaustive_logistic(df, y, candidates)

Finished Var Family: 1
Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
Finished Var Family: 13
Finished Var Family: 14
Finished Var Family: 15
Total Models: 32767


In [164]:
def check_if_features_in(df, features):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    return df[mask]
# print(a_bfs_v2.shape, check_if_features_in(a_bfs_v2, features).shape)
top = check_if_features_in(results, ['x1','x2'])
top['rank'] = top.groupby("Var Number")['Scores'].rank(ascending=False)
top = top[top['rank']<=100]
top.drop('rank', axis=1, inplace=True)
top = top.reset_index(drop=True)

C:\Users\enisb\AppData\Local\Temp\ipykernel_74540\1896177879.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top['rank'] = top.groupby("Var Number")['Scores'].rank(ascending=False)


In [118]:
# df

In [8]:
# np.__version__

In [28]:
%%time
a_orig,b ,c = best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist())

Finished Var Family:  2
Finished Var Family:  3
Finished Var Family:  4
Finished Var Family:  5
CPU times: total: 1min 24s
Wall time: 19.9 s


In [37]:
a_forced, b, c = bs2.best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist(), forced_vars=['x1', 'x2'])
 

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5


In [187]:
%%time
a, b, c = bs2.best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist())
a

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
CPU times: total: 18min 5s
Wall time: 4min 39s


,Var Number,Models,Scores
99,2,x2 x1,17140.282747
98,2,x2 x4,15300.586378
97,2,x2 x3,12622.364252
96,2,x2 x5,11698.923275
95,2,x2 x48,11036.236241
...,...,...,...
304,5,x2 x1 x4 x18 x15,21418.674221
303,5,x2 x1 x4 x15 x45,21418.662492
302,5,x2 x1 x4 x13 x48,21418.614501
301,5,x2 x1 x4 x15 x37,21418.472611


In [32]:
# df

In [44]:
a_bfs_v2, b_bfs_v2, c_bfs_v2= bs7.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=2,  candidates=df.columns[1:].tolist(),  forced_vars=['x1', 'x2'])


In [7]:
%%time
# a_bfs, b_bfs, c_bfs= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
# res_bs1, b_bfs_v2, c_bfs_v2 = bs1.best_subset_bb_logistic(df, y, 100, start=2, stop=15,  candidates=df.columns[1:].tolist())
# a_bfs_v2, b_bfs_v2, c_bfs_v2= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=15,  candidates=df.columns[1:].tolist()) # ,  forced_vars=['x1', 'x2'])
# a_bfs_v2_forced, b_bfs_v2, c_bfs_v2= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=15,  candidates=df.columns[1:].tolist()  ,  forced_vars=['x1', 'x2'])
# a_bfs_v2_forced, b_bfs_v2, c_bfs_v2= bs7.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=15,  candidates=df.columns[1:].tolist()  ,  forced_vars=['x1', 'x2'])
a_bfs_v2_forced, b_bfs_v2, c_bfs_v2 = bs_pkgs.best_subset_bb_logistic(df, y, 100, start=2, stop=15,  candidates=df.columns[1:].tolist()  ,  forced_vars=['x1', 'x2'])
# a_bfs_v3, b_bfs_v3, c_bfs_v3= bs8.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=2,  candidates=df.columns[1:].tolist(),  forced_vars=[['x1', 'x2'], ['x3', 'x4']])
# b_bfs_v2, b_bfs_v3
a_bfs_v2_forced
# a_bfs_v2

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
Finished Var Family: 13
Finished Var Family: 14
Finished Var Family: 15
CPU times: total: 531 ms
Wall time: 172 ms


,Var Number,Models,Scores
0,2,x2 x1,16809.086627
13,3,x2 x1 x4,21162.875388
12,3,x2 x1 x3,18375.780372
11,3,x2 x1 x5,17480.297483
10,3,x2 x1 x12,16813.091352
...,...,...,...
973,14,x2 x1 x4 x3 x5 x7 x6 x14 x9 x15 x13 x8 x10 x11,23399.420311
972,14,x2 x1 x4 x3 x12 x7 x6 x14 x9 x15 x13 x8 x10 x11,22735.386863
971,14,x2 x1 x4 x5 x12 x7 x6 x14 x9 x15 x13 x8 x10 x11,21838.155839
970,14,x2 x1 x3 x5 x12 x7 x6 x14 x9 x15 x13 x8 x10 x11,19051.802959


In [142]:
import re

a_bfs_v2['Models'] = a_bfs_v2['Models'].apply(
    lambda model: " ".join(
        sorted(model.split(" "), key=lambda x: int(re.search(r'\d+', x).group()))
    )
)
a_bfs_v2 = a_bfs_v2.reset_index(drop=True)

In [165]:
# a_bfs_v2_forced.to_csv("bs6.csv")
top.to_csv("top.csv")

In [28]:
import sys
sys.executable

'C:\\Users\\enisb\\python\\envs\\lightgbm_env\\python.exe'

In [161]:
import pandas as pd
import pytest

def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame):
    """Compares two pandas DataFrames, rounding floating-point columns to 2 decimal places."""
    float_cols = df1.select_dtypes(include=['float']).columns
    df1_rounded = df1.copy()
    df2_rounded = df2.copy()
    df1_rounded[float_cols] = df1_rounded[float_cols].round(2)
    df2_rounded[float_cols] = df2_rounded[float_cols].round(2)
    pd.testing.assert_frame_equal(df1_rounded, df2_rounded, check_dtype=False)
    print("Dataset Match")
compare_dataframes(top,a_bfs_v2_forced)
# compare_dataframes(res_bs1,a_bfs_v2)


AssertionError: DataFrame.index are different

DataFrame.index values are different (99.5935 %)
[left]:  RangeIndex(start=0, stop=984, step=1)
[right]: Index([  0,  13,  12,  11,  10,   9,   8,   7,   6,   5,
       ...
       978, 977, 976, 975, 974, 973, 972, 971, 970, 983],
      dtype='int64', length=984)
At positional index 1, first diff: 1 != 13

In [148]:
compare_dataframes(top,a_bfs_v2)
# a_bfs_v2.iloc[0:5,2] = a_bfs_v2.iloc[0:5,2] + 1.02
a_bfs_v2.to_csv("new_algo.csv")
top.to_csv("exeaustive_algo.csv")
# a_bfs_v2
# a_bfs_v3
 

In [29]:
  
both_features_in = a_bfs_v2[a_bfs_v2['Models'].str.contains(r'\bx3\b', regex=True) ] #  & a_bfs_v2['Models'].str.contains(r'\bx2\b', regex=True)]
print(both_features_in.shape, a_bfs_v2.shape)

(54, 3) (229, 3)


In [35]:
 
def check_if_features_in(df, features):
    mask = df['Models'].notna()  # Ensure we exclude NaN values    
    for feature in features:
        mask &= df['Models'].str.contains(rf'\b{feature}\b', regex=True, na=False)
    return df[mask]
print(a_bfs_v2.shape, check_if_features_in(a_bfs_v2, features).shape)
check_if_features_in(a_bfs_v2, features)

(229, 3) (229, 3)


,Var Number,Models,Scores
0,2,x2 x1,17063.643681
28,3,x2 x1 x4,21126.107974
27,3,x2 x1 x3,18558.381970
26,3,x2 x1 x5,17768.059817
25,3,x2 x1 x29,17066.692221
...,...,...,...
133,5,x2 x1 x4 x11 x6,21128.821147
132,5,x2 x1 x4 x27 x6,21128.805017
131,5,x2 x1 x4 x29 x13,21128.790389
130,5,x2 x1 x4 x29 x22,21128.788407


In [59]:
a_forced[
    (a_forced['Models'].str.contains(r'\bx1\b', regex=True) | a_forced['Models'].str.contains(r'\bn2\b', regex=True)) &
    ~a_forced['Models'].str.contains(r'\bx2\b', regex=True)
]

,Var Number,Models,Scores


In [62]:
a_bfs_v2

,Var Number,Models,Scores
0,2,x2 x1,17063.643681
28,3,x2 x1 x4,21126.107974
27,3,x2 x1 x3,18558.381970
26,3,x2 x1 x5,17768.059817
25,3,x2 x1 x29,17066.692221
...,...,...,...
133,5,x2 x1 x4 x11 x6,21128.821147
132,5,x2 x1 x4 x27 x6,21128.805017
131,5,x2 x1 x4 x29 x13,21128.790389
130,5,x2 x1 x4 x29 x22,21128.788407


In [203]:
%%time
a_5, b_5, c_5= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=5,  method="dfs", candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
a_5

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
CPU times: total: 2min 50s
Wall time: 42.3 s


,Var Number,Models,Scores
99,2,x2 x1,17140.282747
98,2,x2 x4,15300.586378
97,2,x2 x3,12622.364252
96,2,x2 x5,11698.923275
95,2,x2 x48,11036.236241
...,...,...,...
304,5,x2 x1 x4 x18 x15,21418.674221
303,5,x2 x1 x4 x15 x45,21418.662492
302,5,x2 x1 x4 x13 x48,21418.614501
301,5,x2 x1 x4 x15 x37,21418.472611


In [214]:
%%time
a_5, b_5, c_5= bs6.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=12,   method="best_first",  candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])


Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
CPU times: total: 3min 20s
Wall time: 53.4 s


In [217]:
%%time
a_5, b_5, c_5= bs7.best_subset_bb_logistic_with_priority(df, y, 100, start=2, stop=12,  method="best_first", candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])


Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
Finished Var Family: 6
Finished Var Family: 7
Finished Var Family: 8
Finished Var Family: 9
Finished Var Family: 10
Finished Var Family: 11
Finished Var Family: 12
CPU times: total: 24 s
Wall time: 29.1 s


In [216]:
b_5

302568

In [184]:
%%time
a,b = bs3.best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
a

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
CPU times: total: 19min 18s
Wall time: 5min 10s


,Var Number,Models,Scores
99,2,x1 x2,17140.282747
98,2,x2 x4,15300.586378
97,2,x2 x3,12622.364252
96,2,x2 x5,11698.923275
95,2,x2 x48,11036.236241
...,...,...,...
304,5,x1 x2 x4 x15 x18,21418.674221
303,5,x1 x2 x4 x15 x45,21418.662492
302,5,x1 x2 x4 x13 x48,21418.614501
301,5,x1 x2 x4 x15 x37,21418.472611


In [185]:
%%time
a,b = bs4.best_subset_bb_logistic(df, y, 100, start=2, stop=5, candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])
a

CPU times: total: 0 ns
Wall time: 0 ns


In [163]:
%%time
 
a_par,b = bs5.best_subset_bb_logistic_parallel(df, y, nbest=100, start=2, stop=15,  candidates=df.columns[1:].tolist()) # , forced_vars=['x1', 'x2'])

Finished Var Size = 2, best_bound = 23402.040
Finished Var Size = 3, best_bound = 23402.040
Finished Var Size = 4, best_bound = 23402.040
Finished Var Size = 5, best_bound = 23402.040
Finished Var Size = 6, best_bound = 23402.040
Finished Var Size = 7, best_bound = 23402.040
Finished Var Size = 8, best_bound = 23402.040
Finished Var Size = 9, best_bound = 23402.040
Finished Var Size = 10, best_bound = 23402.040
Finished Var Size = 11, best_bound = 23402.040
Finished Var Size = 12, best_bound = 23402.040
Finished Var Size = 13, best_bound = 23402.040
Finished Var Size = 14, best_bound = 23402.040
Finished Var Size = 15, best_bound = 23402.040
Total expansions: 224
CPU times: total: 609 ms
Wall time: 484 ms


In [168]:
a_par

,Var Number,Models,Scores
0,2,x1 x2,16809.086627
1,3,x1 x2 x3,18375.780372
2,4,x1 x2 x3 x4,22728.721810
3,5,x1 x2 x3 x4 x5,23396.254730
4,6,x1 x2 x3 x4 x5 x6,23397.524921
5,7,x1 x2 x3 x4 x5 x6 x7,23397.529785
6,8,x1 x2 x3 x4 x5 x6 x7 x8,23398.043450
7,9,x1 x2 x3 x4 x5 x6 x7 x8 x9,23398.573845
8,10,x1 x2 x3 x4 x5 x6 x7 x8 x9 x10,23399.174956
9,11,x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11,23399.335459


In [167]:
a.groupby('Var Number')['Scores'].max()

Var Number
2     16809.086627
3     21162.875388
4     22728.721810
5     23396.254730
6     23398.862804
7     23400.126073
8     23400.742004
9     23401.272679
10    23401.786730
11    23401.950979
12    23402.006755
13    23402.031594
14    23402.037979
15    23402.039790
Name: Scores, dtype: float64

Finished Var Family: 2
Finished Var Family: 3
Finished Var Family: 4
Finished Var Family: 5
CPU times: total: 609 ms
Wall time: 227 ms


,Var Number,Models,Scores
99,2,x2 x1,17140.282747
98,2,x2 x4,15300.586378
97,2,x2 x3,12622.364252
96,2,x2 x5,11698.923275
95,2,x2 x48,11036.236241
...,...,...,...
304,5,x2 x1 x4 x18 x15,21418.674221
303,5,x2 x1 x4 x15 x45,21418.662492
302,5,x2 x1 x4 x13 x48,21418.614501
301,5,x2 x1 x4 x15 x37,21418.472611


In [191]:
b

1000

In [205]:
import numpy as np
import scipy.linalg
import time

# Generate a random symmetric positive-definite matrix I and a random vector v
np.random.seed(42)
n = 500  # Matrix size
I = np.random.rand(n, n)
I = np.dot(I, I.T)  # Make it symmetric positive-definite
v = np.random.rand(n)

# Method 1: Using np.linalg.inv
start_inv = time.time()
I_inv = np.linalg.inv(I)
result_inv = v.T @ I_inv @ v
end_inv = time.time()

# Method 2: Using scipy.linalg.cho_solve
start_cho = time.time()
L = scipy.linalg.cho_factor(I)  # Perform Cholesky factorization
result_cho = v.T @ scipy.linalg.cho_solve(L, v)  # Solve using the factorization
end_cho = time.time()

# Method 3: Using np.linalg.solve
start_solve = time.time()
result_solve = v.T @ np.linalg.solve(I, v)  # Directly solve the system
end_solve = time.time()

# Print results
print("Results:")
print(f"Result using np.linalg.inv: {result_inv}")
print(f"Result using scipy.linalg.cho_solve: {result_cho}")
print(f"Result using np.linalg.solve: {result_solve}")

print("\nTiming:")
print(f"Time using np.linalg.inv: {end_inv - start_inv:.6f} seconds")
print(f"Time using scipy.linalg.cho_solve: {end_cho - start_cho:.6f} seconds")
print(f"Time using np.linalg.solve: {end_solve - start_solve:.6f} seconds")
 

Results:
Result using np.linalg.inv: 395.2539871247952
Result using scipy.linalg.cho_solve: 395.2539871187649
Result using np.linalg.solve: 395.2539871247954

Timing:
Time using np.linalg.inv: 0.019066 seconds
Time using scipy.linalg.cho_solve: 0.011001 seconds
Time using np.linalg.solve: 0.003004 seconds


In [219]:
import heapq

# Create a heap
heap = []
heapq.heappush(heap, 10)
heapq.heappush(heap, 5)
heapq.heappush(heap, 20)
type(heap)

list

In [220]:
 


class Node:
    def __init__(self, key, branches, n, forced_vars=None):
        """
        key: the current subset of variables (excluding 'const')
        branches: how many branches remain
        n: the target subset size or node parameter
        forced_vars: list of variables that must stay in every subset
        """
        if forced_vars is None:
            forced_vars = []

        self.key = key                # full subset (list of strings)
        self.key2 = key[:n]           # partial subset for bounding
        self.branch_id = n - branches + 1
        self.n = n
        self.forced_vars = forced_vars

        self.child = []
        self.key_list = []
        self.has_branches = branches

    def add_children(self):
        """
        Create child nodes by popping one feature at a time
        but skip if that would drop any forced_var from the subset.
        """
        visit = self.has_branches - 1

        for has_branches_new, _ in enumerate(range(visit, 0, -1)):
            child_branch_id = self.n - has_branches_new - 1
            temp = self.key[:]

            # Sanity check: child_branch_id might be out of range
            if child_branch_id < 0 or child_branch_id >= len(temp):
                continue

            removed_feat = temp.pop(child_branch_id)
            # If removing that feature leads to losing forced var, skip
            if removed_feat in self.forced_vars:
                continue

            # Also skip if after removal, any forced var is not in temp
            if not all(fv in temp for fv in self.forced_vars):
                continue

            # If we haven't pruned, then child is valid
            # We also skip if it doesn't actually reduce the subset size
            if len(temp) == self.n - 1:
                # This line in the original code was used to skip 
                # same-size sets, but let's keep it for consistency:
                continue

            new_node = Node(
                temp, 
                has_branches_new + 2, 
                self.n, 
                forced_vars=self.forced_vars
            )
            self.child.append(new_node)
            self.key_list.append(temp)

In [221]:
var = 2
n = var 
root = Node(['x1', 'x2'] , branches=n + 1, n=n, forced_vars=None)

In [224]:
pq = []
heapq.heappush(pq, (-250, root))

In [225]:
pq

[(-250, <__main__.Node at 0x1a544ce6b60>)]